# **道路跟随 - 演示**

在本笔记本中，我们将使用训练好的模型进行预测并控制 **Jetracer** 运动

In [1]:
import sys
sys.path.append('../')
from IPython.display import display
import ipywidgets
import traitlets
import time
from camera import Camera, bgr8_to_jpeg

## **1. 创建 JetRacer 实例**

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [3]:
'''放开阀门限制，完全由交互控件控制速度'''
car.throttle_gain = 1
car.steering_gain = -1
'''设置前轮默认偏移量'''
car.steering_offset= -0.037 

#### 1.1 **创建交互滑块来控制 Jetracer**

``throttle_slider``  手动设置油门大小

``steering_slider`` 手动设置转角大小



In [4]:
throttle_switch = ipywidgets.FloatSlider(min=0, max=1, step=1, value=0) # 油门
steering_slider = ipywidgets.FloatSlider(min=-0.8, max=0.8, step=0.1, value=0) # 转角
car_safe_button = ipywidgets.Button(description='SAFE',button_style='warning', layout=ipywidgets.Layout(width='300px', height='28px'))

def car_safe(change):
    car.throttle = 0
    car.steering = 0
    throttle_switch.value = 0
    steering_slider.value = 0

jetracer_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*24+' JetRacer 控制 '+'-'*40),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('停止/启动'),ipywidgets.Label('转角')
        ]),
        ipywidgets.VBox([
            throttle_switch, steering_slider
        ])
    ]),
    car_safe_button
])

#display(jetracer_control_widgets)

#### 1.2 **控件与JetRacer 实例关联**

In [5]:
#traitlets.dlink((throttle_slider, 'value'), (car, 'throttle'), transform=lambda x: x)
traitlets.dlink((steering_slider, 'value'), (car, 'steering'), transform=lambda x: x)
car_safe_button.on_click(car_safe)

## **2. 创建 Camera 实例**
设置摄像头获取的图像为较大的图像，之后再resize至 **resnet18** 支持的输入 **224 X 224**

摄像头获取的图像大小应适用于相机校准的输入（默认为608*342）

In [6]:
image_width = 448#224
image_height = 448#224
display_width = 224
display_height = 224
camera_fps = 30

In [7]:
camera = Camera(width=image_width, height=image_height, fps=camera_fps)

## **3. 加载模型**

``target_number``  设置输出目标数

> 确保已经完成训练并生成 ``best_steering_model_xy.pth``

In [8]:
model_path = 'model/best_follow_xy1_double_target_50epochs_trt.pth'
target_number = 2

## 3.1 **加载一般模型** (二选一)

In [ ]:
import torchvision
import torch

# 定义模型结构
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2*target_number)

# 加载参数
model.load_state_dict(torch.load(model_path))

# 加载到GPU
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

## 3.2 **加载 TensonRT 模型** (二选一)

In [10]:
import torch
from torch2trt import TRTModule

model = TRTModule()
model.load_state_dict(torch.load(model_path))
device = torch.device('cuda')

## **4. 创建预处理功能**

In [11]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

#### **将从相机获取的数据变换为模型支持的格式。**

1. 从HWC布局转换到CHW布局
2. 使用与训练时一样的参数归一化， 从**[0， 255]** 重映射到 **[0，1]**
3. 将数据从**CPU**传输到**GPU**
4. 添加 batch 尺寸

In [13]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    #image = transforms.functional.resize(image,(224, 224))
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## **5. 设置变道功能函数**
#### **5.1 定义变道功能按钮**

In [14]:
lane_left_widgets = ipywidgets.Button(description='LEFT',button_style='warning')
lane_right_widgets = ipywidgets.Button(description='RIGHT',button_style='success')

change_lane_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*25+' change lane '+'-'*25),
    ipywidgets.HBox([lane_left_widgets, lane_right_widgets])
])

#display(change_lane_widgets)

#### **5.2 定义变道功能**

>**current_lane** 设置默认当前车道

In [15]:
current_lane = 'left'

def change_left_lane(change):
    global current_lane
    current_lane = 'left'
    lane_left_widgets.button_style = 'warning'
    lane_right_widgets.button_style = 'success'
    
def change_right_lane(change):
    global current_lane
    current_lane = 'right'
    lane_left_widgets.button_style = 'success'
    lane_right_widgets.button_style = 'warning'
    
lane_left_widgets.on_click(change_left_lane)
lane_right_widgets.on_click(change_right_lane)

## **6. 创建控制功能**
### **6.1 PID控制转角**

In [16]:
class SteeringPID(traitlets.HasTraits):
    Kp_bias = traitlets.Float()# 比例增益偏置
    Kp_factor = traitlets.Float()# 比例增益系数
    Ki = traitlets.Float()# 积分增益
    Kd = traitlets.Float()# 微分增益
    bias = traitlets.Float()# 偏置增益
    
    def __init__(self, max_out, min_out, windup_guard):
        self.max_out = max_out # PID算法最大的输出 
        self.min_out = min_out # PID算法最小的输出
        self.integral_error = 0 # 累加误差
        self.last_error = 0 # 上一次的误差
        self.current_time = time.time() #当前时刻
        self.last_time = self.current_time
        self.setPoint = 0.0
        self.windup_guard = windup_guard # 最大积分项最大
        self.new_Kp = 0.0
    
    def calculate(self, pv):
        '''输入误差，计算PID'''
        error = self.setPoint - pv
        
        self.current_time = time.time()
        delta_time = self.current_time - self.last_time
        
        self.new_Kp = (error*error)*self.Kp_factor + self.Kp_bias
        P_out = error* self.new_Kp # 比例项
        
        self.integral_error += error * delta_time
        if self.integral_error < -self.windup_guard:
            self.integral_error = -self.windup_guard
        elif self.integral_error > self.windup_guard:
            self.integral_error = self.windup_guard
        I_out = self.Ki * self.integral_error # 积分项
        
        derivative = (error - self.last_error) / delta_time # 微分项
        D_out = self.Kd * derivative
        
        PID_out = P_out + I_out + D_out + self.bias # 计算最终PID输出
        
        # 判断上下边界
        if PID_out > self.max_out:
            PID_out = self.max_out
        elif PID_out < self.min_out:
            PID_out = self.min_out
            
        self.last_error = error #保存当前损失，供下次计算
        self.last_time = self.current_time # 保存上一次时间戳
        
        return PID_out
    
    def reset(self):
        self.integral_error = 0
        self.last_error = 0
        self.current_time = time.time() #当前时刻
        self.last_time = self.current_time
        self.setPoint = 0.0
        return 'success reset'
    def get_value(self):
        return self.new_Kp, self.Ki, self.Kd
      
'''实例化PID控制器，并设置最大转角'''
steering_controller = SteeringPID(0.9, -0.9, 0.8)

### **6.2 创建交互控件进行PID控制**
``steering_kp_slider`` 设置转向kP（比例参数），该参数与``error``成线性关系，当产生误差且误差改变时，该值越大表示越快到达目标位置（越快减小误差），但是设置太大，容易造成在目标位置附近震荡

``steering_kd_slider`` 设置转向kD（微分参数），该参数能够对到达目标位置（减小误差）的速度（对error做微分，得到的是速度）做出响应。该值能够对P算法产生的结果进行缓冲（减小震荡）

``steering_bias_slider`` 设置偏置项， 如果运行中，总是过于偏向某一边，则应该修改该参数，使Jetracer沿着中心线移动

In [17]:
import time

steering_Kp_bias_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.32) # 比例增益_偏置项
steering_Kp_factor_slider = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=1.10) # 比例增益_系数
steering_Ki_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0) # 积分增益
steering_Kd_slider = ipywidgets.FloatSlider(min=-0.6, max=0.6, step=0.01, value=0.02) # 微分增益
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.00) # 偏置增益
pid_reset_button = ipywidgets.Button(description='PID RESET',button_style='warning')

def pid_reset(change):
    global steering_controller
    steering_controller.reset()


steering_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*38+' 转角PID控制参数 '+'-'*38),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('Kp(比例增益-偏置项)'),ipywidgets.Label('Kp(比例增益-系数)'),
            ipywidgets.Label('Ki(积分增益)'),
            ipywidgets.Label('Kd(微分增益)/10'),ipywidgets.Label('bias(偏置增益)'),
        ]),
        ipywidgets.VBox([
            steering_Kp_bias_slider, steering_Kp_factor_slider,
            steering_Ki_slider, steering_Kd_slider, 
            steering_bias_slider
        ])
    ]),
    pid_reset_button
])


time.sleep(1)
traitlets.dlink((steering_Kp_bias_slider, 'value'), (steering_controller, 'Kp_bias'), transform=lambda x: x)
traitlets.dlink((steering_Kp_factor_slider, 'value'), (steering_controller, 'Kp_factor'), transform=lambda x: x)
traitlets.dlink((steering_Ki_slider, 'value'), (steering_controller, 'Ki'), transform=lambda x: x)
traitlets.dlink((steering_Kd_slider, 'value'), (steering_controller, 'Kd'), transform=lambda x: x/10)
traitlets.dlink((steering_bias_slider, 'value'), (steering_controller, 'bias'), transform=lambda x: x)
pid_reset_button.on_click(pid_reset)

### **6.3 判断直线及弯道**

In [18]:
class SpeedControl(traitlets.HasTraits):
    straight_throttle = traitlets.Float()
    curve_throttle = traitlets.Float()
    angle_threshold = traitlets.Float()
    min_tracking_number = traitlets.Int()
    
    def __init__(self, max_out, min_out):
        self.errors = []
        self.curve_straight = 'curve'
        self.out = 0.0
        self.max_out = max_out
        self.min_out = min_out
    def calculate(self, angle):
        self.errors.append(angle) 
        #默认为低速模式
        self.curve_straight = 'curve'
        self.out = self.curve_throttle
        #如果最近三次记录的误差值都在直道阈值内，则说明可以加速
        if len(self.errors) > self.min_tracking_number:
            del(self.errors[0])
            #满足判断要求
            if all(_ > -self.angle_threshold for _ in self.errors) and all(_ < self.angle_threshold for _ in self.errors):
                self.curve_straight = 'straight'
                self.out = self.straight_throttle
        
        if self.out > self.max_out:
            self.out = self.max_out
        elif self.out < self.min_out:
            self.out = self.min_out
        return self.out, self.curve_straight
    
    def reset(self):
        del(self.errors[:])
    def get_value(self):
        return self.straight_throttle, self.curve_throttle
    def get_threshold(self):
        return self.angle_threshold

throttle_controller = SpeedControl(0.25, 0.1)

### **6.4 创建速度控制器的交互控件**

In [ ]:
straight_throttle_sliber = ipywidgets.FloatSlider(min=0.10, max=0.25, step=0.01, value=0.18)
curve_throttle_sliber = ipywidgets.FloatSlider(min=0.10, max=0.25, step=0.01, value=0.14)
angle_threshold_sliber = ipywidgets.FloatSlider(min=0.01, max=0.9, step=0.02, value=0.1)
min_tracking_number_sliber = ipywidgets.IntSlider(min=0, max=50, step=1, value=10)

traitlets.dlink((straight_throttle_sliber, 'value'), (throttle_controller, 'straight_throttle'), transform=lambda x: x)
traitlets.dlink((curve_throttle_sliber, 'value'), (throttle_controller, 'curve_throttle'), transform=lambda x: x)
traitlets.dlink((angle_threshold_sliber, 'value'), (throttle_controller, 'angle_threshold'), transform=lambda x: x)
traitlets.dlink((min_tracking_number_sliber, 'value'), (throttle_controller, 'min_tracking_number'), transform=lambda x: x)

throttle_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*30+' 纵轴油门控制参数 '+'-'*30),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('直道最高速度'),ipywidgets.Label('弯道最高速度'),
            ipywidgets.Label('判断直道/弯道阈值'),ipywidgets.Label('最小稳定跟踪次数')
        ]),
        ipywidgets.VBox([
            straight_throttle_sliber, curve_throttle_sliber, angle_threshold_sliber, min_tracking_number_sliber
        ])
    ])
])

## **7. 可视化预测结果和运行状态**

#### **7.1 定义交互控件显示预测结果及运行状态**

``live_widget`` 显示图片数据

``live_steering_slider`` 显示 JetRacer 当前的转角 [-1, 1]

``live_throttle_slider`` 显示 JetRacer 当前的前进方向 [-1, 1]

> 仅仅展示数值，手动修改并不会影响程序中相关变量的值 



In [21]:
live_image_widget = ipywidgets.Image(format='jpeg', width=display_width, height=display_height)

live_steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0)
live_throttle_slider = ipywidgets.FloatSlider(min=0, max=1.0)

prediction_text = ipywidgets.Textarea()

live_widget = ipywidgets.VBox([
    ipywidgets.Label('-'*65+'  LIVE  '+'-'*65),
    ipywidgets.HBox([
        live_image_widget,
        ipywidgets.VBox([
            prediction_text,
            ipywidgets.HBox([
                ipywidgets.VBox([
                    ipywidgets.Label('live_steering'), ipywidgets.Label('live_throttle')
                ]),
                ipywidgets.VBox([
                    live_steering_slider,live_throttle_slider
                ])
            ])
        ])
    ])
    
])

#### **7.2 创建角度阈值画图函数**

In [ ]:
def draw_angle_threshold(image, threshold):
    h, w = image.shape[:2]
    origin_point = (int(w/2), h-1)
    if threshold < np.arctan2(w/2, h) and threshold > np.arctan2(-w/2, h):
        left_point = (int(w/2 + h*np.tan(-1*threshold)), 0)
        right_point = (int(w/2 + h * np.tan(threshold)), 0)
        image = cv2.line(image, origin_point, left_point, (0, 0, 255), 2)
        image = cv2.line(image, origin_point, right_point, (0, 0, 255), 2)
    else:
        left_point = (0, int(h + w / 2 / np.tan(-1*threshold)))
        right_point = (w-1, int(h - w / 2 / np.tan(threshold)))
        image = cv2.line(image, origin_point, left_point, (0, 255, 0), 2)
        image = cv2.line(image, right_point, origin_point, (0, 255, 0), 2)
    return image

#### **7.3 定义画图功能函数**

In [22]:
def drawing(image, targets, fps, angle_threshold):
    h, w = image.shape[:2]
    # 画辅助线
    image = cv2.line(image, (int(w/2), 0), (int(w/2), h-1), (20, 150, 20),2)
    image = cv2.line(image, (0, int(h/2)), (w-1, int(h/2)), (20, 150, 20),2)
    # 画弯道阈值线
    image = draw_angle_threshold(image, angle_threshold)
    # 画目标点
    for target in targets:
        x, y, color = target
        circle_x = int(w * (x + 1.0)/2.0)
        circle_y = int(h * (y + 1.0)/2.0)
        image = cv2.circle(image, (circle_x, circle_y), 6, color, 2)
    # 显示文字信息
    #for index, text in enumerate(texts):
    image = cv2.putText(image, fps, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return image

## **8. 创建 “处理” 函数**

#### **8.1 定义处理函数**

1. 预处理相机图像
2. 执行神经网络
3. 计算近似转向值
4. 使用比例/微分控制（PD）控制电机

In [23]:
steering_controller.reset() # 转角控制器初始化
throttle_controller.reset() # 速度控制器初始化

def execute(change):
    global Steering_controller, current_lane
    first_time = time.time()
    image = change['new']
    
    '''原图校准'''
    #image = correctImage(image,coefficient_group)
    t1 = time.time()

    '''resize'''
    image = cv2.resize(image, (224, 224))
    t2 = time.time()
    
    '''模型预测'''
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    t3 = time.time()
    
    '''获取坐标'''
    x_1, y_1 = xy[0], xy[1]
    x_2, y_2 = xy[2], xy[3]
    transform_y_1 = y_1/2.0 + 0.5 #映射到[0: 1]
    transform_y_2 = y_2/2.0 + 0.5 
    
    '''计算转角值'''
    #arctan2得到的是目标点与纵向中轴线的夹角的角度。
    if current_lane == 'left':
        #angle = np.arctan2(x_1, transform_y_1)
        deviation = x_1
    elif current_lane == 'right':
        #angle = np.arctan2(x_2, transform_y_2)
        deviation = x_2
    pred_steering = -1 * steering_controller.calculate(deviation)
    
    end_time = time.time()
    
    '''根据道路类型，计算油门值'''
    if current_lane == 'left':
        transform_y = y_1 + 1.0
        angle = np.arctan2(x_1, transform_y)
    elif current_lane == 'right':
        transform_y = y_2 + 1.0
        angle = np.arctan2(x_2, transform_y)
    #返回油门值及道路类型
    pred_throttle, curve_straight = throttle_controller.calculate(angle)
        
    '''进行控制输出'''
    car.steering = pred_steering
    car.throttle = pred_throttle * throttle_switch.value
    
    end_time = time.time()
    
    '''可视化检测结果，及运行状态'''
    # 图里画出目标点及帧率
    targets = [
        [x_1, y_1, (0, 0, 255) if current_lane == 'left' else (0, 255, 0)],
        [x_2, y_2,(0, 0, 255) if current_lane == 'right' else (0, 255, 0)]
    ]
    image = drawing(image, targets, 'FPS: %.2f'%(1/(end_time-first_time)), throttle_controller.get_threshold())
    
    #文本框显示信息
    
    texts = [
        'FPS: %.2f'%(1/(end_time-first_time)), # 显示FPS
        'target_1: (%.2f, %.2f) %s'%(x_1, y_1, 'follow' if current_lane == 'left' else ''), # 显示预测的第一个目标坐标
        'target_2: (%.2f, %.2f) %s'%(x_2, y_2, 'follow' if current_lane == 'right' else ''), # 显示预测的第二个目标坐标
        'steering: Kp:%.3f Ki:%.3f Kd:%.3f'%(steering_controller.get_value()),
        'throttle: max: %.2f min: %.2f'%(throttle_controller.get_value()),
        'pred_steering: %.2f'%(pred_steering), 
        'pred_throttle: %.2f'%(pred_throttle), 
        'state: %s %s'%(curve_straight, 'Speed up'if curve_straight == 'straight' else 'slow down')
    ]
    new_text = ''
    for text in texts:
        new_text += text
        new_text += '\n'
    prediction_text.value = new_text
    #滑动条显示结果
    live_steering_slider.value = pred_steering
    #live_throttle_slider.value = pred_throttle
    
    live_image_widget.value = bgr8_to_jpeg(image)
    
#camera.observe(execute, names='value')
execute({'new': camera.value}) #试跑一遍

#### **8.2 定义开始按钮**

In [24]:
start_button = ipywidgets.Button(description='START',button_style='danger', layout=ipywidgets.Layout(width='300px', height='28px'))
start_widget = ipywidgets.VBox([
    ipywidgets.Label('-'*50+' START RUN '+'-'*50),
    start_button
])

def start_execute(change):
    global steering_controller, steering_controller
    if start_button.description == 'START':
        steering_controller.reset() # 转角控制器初始化
        throttle_controller.reset() # 速度控制器初始化
        camera.observe(execute, names='value')
        start_button.description = 'STOP'
        start_button.button_style='warning'
    else:
        camera.unobserve(execute, names='value')
        start_button.description = 'START'
        start_button.button_style='danger'
        
start_button.on_click(start_execute)

## **9. 可视化检测结果及显示参数调教模块**
在该部分，可进行控制参数调教，及检验模型预测效果

| 部件 | 功能 |
|--------|-------------|
| 油门 | 实际控制JetRacer 纵轴运动 |
| 转角 | 实际控制JetRacer 横轴运动 |
| SAFE | JetRacer 归位（急停）|
| Kp(比例增益) | 控制PID 比例增益 |
| Ki(积分增益) | 控制PID 积分增益 |
| Kd(微分增益) | 控制PID 微分增益 |
| bias(偏置增益) | 控制PID 偏置增益 |
| live_steering | 显示此时的转角 |
| live_throttle | 显示此时的油门 |
| LEFT/RIGHT | 控制左右变道 |
| START | 开始运动开关 |

In [25]:
display(
    ipywidgets.VBox([
        ipywidgets.HBox([
            ipywidgets.VBox([jetracer_control_widgets,throttle_control_widgets]),
            steering_control_widgets
        ]),
        live_widget,
        change_lane_widgets,
        start_widget
    ]) 
)

## **10. 释放Camera及JetRacer实例**

In [60]:
#car.stop()
camera.stop()